## Package

In [1]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os.path
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.models import Sequential
from keras.models import Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

from keras.utils import plot_model
from keras import regularizers
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
#from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_uniform
import visualkeras
from keras.optimizers import Adam

from IPython.display import Markdown, display
import seaborn as sns

# from lib import proc_img
# from lib import create_gen

def printmd(string):
    #Print with Markdowns    
    display(Markdown(string))

In [2]:
%load_ext autotime
RANDOM_STATE = 42

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

time: 1.49 s


In [4]:
seed = 7
np.random.seed(seed)

time: 69.5 ms


In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

time: 61.8 ms


In [6]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

time: 6.31 s


In [7]:
tf.test.is_built_with_cuda()

True

time: 2.32 ms


In [8]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

time: 7.45 s


In [9]:
x_train, x_valid, y_train, y_valid = train_test_split(train_images, train_labels, test_size=0.2, shuffle= True)

time: 269 ms


In [10]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

time: 2.74 ms


In [11]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

time: 94 ms


In [ ]:
dense_sizes_1 = [64,128,256,512]
dense_sizes_2 = [64,128,256,512]
learning_rates_1 = [1e-3,1e-1, 1e-2, 1e-4, 1e-5]

s1_filter_sizes_1 = [16,32,64,128,256,512]

s2_filter_sizes_1 = [16,32,64,128,256,512]
s2_filter_sizes_2 = [16,32,64,128,256,512]
s2_filter_sizes_3 = [128,256,512,1024,2048]

s3_filter_sizes_1 = [16,32,64,128,256,512]
s3_filter_sizes_2 = [16,32,64,128,256,512]
s3_filter_sizes_3 = [128,256,512,1024,2048]

s4_filter_sizes_1 = [16,32,64,128,256,512]
s4_filter_sizes_2 = [16,32,64,128,256,512]
s4_filter_sizes_3 = [128,256,512,1024,2048]

s5_filter_sizes_1 = [16,32,64,128,256,512]
s5_filter_sizes_2 = [16,32,64,128,256,512]
s5_filter_sizes_3 = [128,256,512,1024,2048]

dp_sizes_1 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
dp_sizes_2 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]

wd_sizes_1=[0.0,0.1, 0.001, 0.0001]
wd_sizes_2=[0.0,0.1, 0.001, 0.0001]

opt_layer_1=0.001
opt_layer_2=64
opt_layer_3=64
opt_layer_4=16
opt_layer_5=16
opt_layer_6=16
opt_layer_7=128
opt_layer_8=16
opt_layer_9=16
opt_layer_10=128
opt_layer_11=16
opt_layer_12=16
opt_layer_13=128
opt_layer_14=16
opt_layer_15=16
opt_layer_16=128
opt_layer_17=0.0
opt_layer_18=0.0
opt_layer_19=0.0
opt_layer_20=0.0

temp=0
test_acc=0
combinations=0
flag=0
for wd_size_2 in wd_sizes_2 :
    for wd_size_1 in wd_sizes_1 :
        for dp_size_2 in dp_sizes_2 :
            for dp_size_1 in dp_sizes_1 :
                for s5_filter_size_3 in s5_filter_sizes_3:
                    for s5_filter_size_2 in s5_filter_sizes_2:
                        for s5_filter_size_1 in s5_filter_sizes_1:
                            for s4_filter_size_3 in s4_filter_sizes_3:
                                for s4_filter_size_2 in s4_filter_sizes_2:
                                    for s4_filter_size_1 in s4_filter_sizes_1 :
                                        for s3_filter_size_3 in s3_filter_sizes_3:
                                            for s3_filter_size_2 in s3_filter_sizes_2:
                                                for s3_filter_size_1 in s3_filter_sizes_1:
                                                    for s2_filter_size_3 in s2_filter_sizes_3:
                                                        for s2_filter_size_2 in s2_filter_sizes_2:
                                                            for s2_filter_size_1 in s2_filter_sizes_1:
                                                                for s1_filter_size_1 in s1_filter_sizes_1:
                                                                    for dense_size_2 in dense_sizes_2:
                                                                        for dense_size_1 in dense_sizes_1:
                                                                            for learning_rate_1 in learning_rates_1:
                                                                                print(learning_rate_1,dense_size_1,dense_size_2,s1_filter_size_1,s2_filter_size_1,s2_filter_size_2,s2_filter_size_3,s3_filter_size_1,s3_filter_size_2,s3_filter_size_3,s4_filter_size_1,s4_filter_size_2,s4_filter_size_3,s5_filter_size_1,s5_filter_size_2,s5_filter_size_3,dp_size_1,dp_size_2,wd_size_1,wd_size_2)
                                                                                input_shape=(32, 32, 3)
                                                                                X_input = Input(input_shape)

                                                                                X = ZeroPadding2D((1, 1))(X_input)

                                                                                X = Conv2D(s1_filter_size_1, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X = BatchNormalization(axis=3, name='bn_conv1')(X)
                                                                                X = Activation('relu')(X)
                                                                                X = MaxPooling2D((3, 3), strides=(2, 2))(X)

                                                                                X = convolutional_block(X, f=3, filters=[s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='a', s=1)
                                                                                X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='b')
                                                                                X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='c')

                                                                                X = convolutional_block(X, f=3, filters=[s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='a', s=2)
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='b')
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='c')
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='d')

                                                                                X = convolutional_block(X, f=3, filters=[s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='a', s=2)
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='b')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='c')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='d')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='e')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='f')

                                                                                X = X = convolutional_block(X, f=3, filters=[s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='a', s=2)
                                                                                X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='b')
                                                                                X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='c')

                                                                                X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

                                                                                X= Dropout(dp_size_1)(X)
                                                                                X = Flatten()(X)
                                                                                X=Dense(dense_size_1, activation='relu',kernel_regularizer=regularizers.l2(wd_size_1), name='fc1',kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X=Dense(dense_size_2, activation='relu',kernel_regularizer=regularizers.l2(wd_size_2), name='fc2',kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X= Dropout(dp_size_2)(X)
                                                                                X = Dense(10,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(X)

                                                                                model = Model(inputs=X_input, outputs=X, name='ResNet50')

                                                                                my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]
                                                                                opt = Adam(learning_rate=learning_rate_1)

                                                                                model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                                                                                                optimizer=opt,
                                                                                                                metrics=['accuracy'])
                                                                                model.fit(x_train,y_train,
                                                                                          batch_size=64,
                                                                                          epochs=100,
                                                                                          validation_data=(x_valid,y_valid),
                                                                                          callbacks=[my_callbacks])
                                                                                

                                                                                _,test_acc = model.evaluate(x_valid, y_valid)
                                                                                print("Current Acc: ",test_acc)
                                                                                print("Best Acc: ",temp)
                                                                                combinations=combinations+1
                                                                                if flag==0 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_1 = learning_rate_1
                                                                                        temp=test_acc
                                                                                    learning_rates_1 = [opt_layer_1]

                                                                                elif flag==1 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_2 = dense_size_1
                                                                                        temp=test_acc
                                                                                    dense_sizes_1 = [opt_layer_2]

                                                                                elif flag==2 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_3 = dense_size_2
                                                                                        temp=test_acc
                                                                                    dense_sizes_2 = [opt_layer_3]

                                                                                elif flag==3 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_4 = s1_filter_size_1
                                                                                        temp=test_acc
                                                                                    s1_filter_sizes_1 = [opt_layer_4]
                                                                                    
                                                                                elif flag==4 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_5 = s2_filter_size_1
                                                                                        temp=test_acc
                                                                                    s2_filter_sizes_1 = [opt_layer_5]
                                                                                    
                                                                                elif flag==5 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_6 = s2_filter_size_2
                                                                                        temp=test_acc
                                                                                    s2_filter_sizes_2 = [opt_layer_6]
                                                                                    
                                                                                elif flag==6 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_7 = s2_filter_size_3
                                                                                        temp=test_acc
                                                                                    s2_filter_sizes_3 = [opt_layer_7]

                                                                                elif flag==7 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_8 = s3_filter_size_1
                                                                                        temp=test_acc
                                                                                    s3_filter_sizes_1 = [opt_layer_8]
                                                                                    
                                                                                elif flag==8 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_9 = s3_filter_size_2
                                                                                        temp=test_acc
                                                                                    s3_filter_sizes_2 = [opt_layer_9]
                                                                                    
                                                                                elif flag==9 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_10 = s3_filter_size_3
                                                                                        temp=test_acc
                                                                                    s3_filter_sizes_3 = [opt_layer_10]
                                                                                    
                                                                                elif flag==10 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_11 = s4_filter_size_1
                                                                                        temp=test_acc
                                                                                    s4_filter_sizes_1 = [opt_layer_11]
                                                                                    
                                                                                elif flag==11 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_12 = s4_filter_size_2
                                                                                        temp=test_acc
                                                                                    s4_filter_sizes_2 = [opt_layer_9]
                                                                                    
                                                                                elif flag==12 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_13 = s4_filter_size_3
                                                                                        temp=test_acc
                                                                                    s4_filter_sizes_3 = [opt_layer_13]
                                                                                    
                                                                                elif flag==13 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_14 = s5_filter_size_1
                                                                                        temp=test_acc
                                                                                    s5_filter_sizes_1 = [opt_layer_14]
                                                                                    
                                                                                elif flag==14 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_15 = s5_filter_size_2
                                                                                        temp=test_acc
                                                                                    s5_filter_sizes_2 = [opt_layer_15]
                                                                                    
                                                                                elif flag==15 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_16 = s5_filter_size_3
                                                                                        temp=test_acc
                                                                                    s5_filter_sizes_3 = [opt_layer_16]
                                                                                    
                                                                                elif flag==16 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_17 = dp_size_1
                                                                                        temp=test_acc
                                                                                    dp_sizes_1 = [opt_layer_17]
                                                                                    
                                                                                elif flag==17 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_18 = dp_size_2
                                                                                        temp=test_acc
                                                                                    dp_sizes_2 = [opt_layer_18]
                                                                                    
                                                                                elif flag==18 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_19 = wd_size_1
                                                                                        temp=test_acc
                                                                                    wd_sizes_1 = [opt_layer_19]
                                                                                    
                                                                                elif flag==19 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_20 = wd_size_2
                                                                                        temp=test_acc
                                                                                    wd_sizes_2 = [opt_layer_20]
                                                                            flag=1
                                                                            test_acc=0
                                                                        flag=2
                                                                        test_acc=0
                                                                    flag=3
                                                                    test_acc=0
                                                                flag=4
                                                                test_acc=0
                                                            flag=5
                                                            test_acc=0
                                                        flag=6
                                                        test_acc=0
                                                    flag=7
                                                    test_acc=0
                                                flag=8
                                                test_acc=0
                                            flag=9
                                            test_acc=0
                                        flag=10
                                        test_acc=0  
                                    flag=11
                                    test_acc=0
                                flag=12
                                test_acc=0
                            flag=13
                            test_acc=0
                        flag=14
                        test_acc=0
                    flag=15
                    test_acc=0
                flag=16
                test_acc=0
            flag=17
            test_acc=0
        flag=18
        test_acc=0
    flag=19
    test_acc=0
    
print("######################################################################################################")
print("Trials: ",combinations)
print("Optimal Validation Accuracy",temp)
print("The Final Optimal Values for all block is : ")
print("learning rate: ",opt_layer_1,"\ndense_size_1: ",opt_layer_2,"\ndense_size_2: ",opt_layer_3,"\ns1_filter_size_1: ",opt_layer_4,"\ns2_filter_size_1: ",opt_layer_5,"\ns2_filter_size_2: ",opt_layer_6,"\ns2_filter_size_3: ",opt_layer_7,"\ns3_filter_size_1: ",opt_layer_8,"\ns3_filter_size_2: ",opt_layer_9,"\ns3_filter_size_3: ",opt_layer_10,"\ns4_filter_size_1: ",opt_layer_11,"\ns4_filter_size_2: ",opt_layer_12,"\ns4_filter_size_3: ",opt_layer_13,"\ns5_filter_size_1: ",opt_layer_14,"\ns5_filter_size_2: ",opt_layer_15,"\ns5_filter_size_3: ",opt_layer_16,"\ndp_size_1: ",opt_layer_17,"\ndp_size_2: ",opt_layer_18,"\nwd_size_1: ",opt_layer_19,"\nwd_size_2: ",opt_layer_20)                      

0.001 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 58s 29ms/step - loss: 2.2057 - accuracy: 0.1732 - val_loss: 1.8601 - val_accuracy: 0.3061
Epoch 2/100
625/625 [==============================] - 17s 27ms/step - loss: 1.7525 - accuracy: 0.3448 - val_loss: 1.7404 - val_accuracy: 0.3638
Epoch 3/100
625/625 [==============================] - 17s 26ms/step - loss: 1.5917 - accuracy: 0.4123 - val_loss: 1.8658 - val_accuracy: 0.3398
Epoch 4/100
625/625 [==============================] - 16s 26ms/step - loss: 1.4801 - accuracy: 0.4559 - val_loss: 2.0395 - val_accuracy: 0.3124
Epoch 5/100
625/625 [==============================] - 16s 26ms/step - loss: 1.4031 - accuracy: 0.4907 - val_loss: 1.5112 - val_accuracy: 0.4617
Epoch 6/100
625/625 [==============================] - 17s 27ms/step - loss: 1.3403 - accuracy: 0.5168 - val_loss: 1.7265 - val_accuracy: 0.3951
Epoch 7/100
625/625 [==============================] - 17s 

625/625 [==============================] - 17s 28ms/step - loss: 1.1871 - accuracy: 0.5698 - val_loss: 1.5623 - val_accuracy: 0.4548
Epoch 29/100
625/625 [==============================] - 18s 28ms/step - loss: 1.1755 - accuracy: 0.5787 - val_loss: 1.4272 - val_accuracy: 0.5023
Epoch 30/100
625/625 [==============================] - 17s 28ms/step - loss: 1.1681 - accuracy: 0.5818 - val_loss: 1.4168 - val_accuracy: 0.4996
Epoch 31/100
625/625 [==============================] - 17s 28ms/step - loss: 1.1471 - accuracy: 0.5881 - val_loss: 1.4111 - val_accuracy: 0.5034
Epoch 32/100
625/625 [==============================] - 17s 28ms/step - loss: 1.1220 - accuracy: 0.5974 - val_loss: 1.4024 - val_accuracy: 0.5068
Epoch 33/100
625/625 [==============================] - 17s 28ms/step - loss: 1.1223 - accuracy: 0.5968 - val_loss: 1.4428 - val_accuracy: 0.4934
Epoch 34/100
625/625 [==============================] - 18s 28ms/step - loss: 1.1050 - accuracy: 0.6018 - val_loss: 1.3957 - val_accuracy

Epoch 43/100
625/625 [==============================] - 17s 27ms/step - loss: 1.9231 - accuracy: 0.2789 - val_loss: 1.9332 - val_accuracy: 0.2803
Epoch 44/100
625/625 [==============================] - 17s 27ms/step - loss: 1.9139 - accuracy: 0.2828 - val_loss: 1.9215 - val_accuracy: 0.2842
Epoch 45/100
625/625 [==============================] - 17s 27ms/step - loss: 1.8994 - accuracy: 0.2873 - val_loss: 1.9092 - val_accuracy: 0.2821
Epoch 46/100
625/625 [==============================] - 17s 27ms/step - loss: 1.8858 - accuracy: 0.2966 - val_loss: 1.8981 - val_accuracy: 0.2860
Epoch 47/100
625/625 [==============================] - 17s 27ms/step - loss: 1.8801 - accuracy: 0.2976 - val_loss: 1.8940 - val_accuracy: 0.2960
Epoch 48/100
625/625 [==============================] - 17s 27ms/step - loss: 1.8652 - accuracy: 0.2987 - val_loss: 1.8788 - val_accuracy: 0.2996
Epoch 49/100
625/625 [==============================] - 17s 27ms/step - loss: 1.8563 - accuracy: 0.3050 - val_loss: 1.8701 -

625/625 [==============================] - 17s 27ms/step - loss: 1.5525 - accuracy: 0.4264 - val_loss: 1.6269 - val_accuracy: 0.3966
Epoch 100/100
313/313 [==============================] - 3s 9ms/step - loss: 1.6270 - accuracy: 0.3993
Current Acc:  0.3993000090122223
Best Acc:  0.527899980545044
0.001 128 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 23s 29ms/step - loss: 2.1990 - accuracy: 0.1696 - val_loss: 2.0593 - val_accuracy: 0.2684
Epoch 2/100
625/625 [==============================] - 17s 26ms/step - loss: 1.7674 - accuracy: 0.3369 - val_loss: 1.8178 - val_accuracy: 0.3407
Epoch 3/100
625/625 [==============================] - 16s 26ms/step - loss: 1.5912 - accuracy: 0.4109 - val_loss: 1.5793 - val_accuracy: 0.4181
Epoch 4/100
625/625 [==============================] - 17s 26ms/step - loss: 1.4836 - accuracy: 0.4584 - val_loss: 1.6286 - val_accuracy: 0.4282
Epoch 5/100
625/625 [=============================

Epoch 7/100
625/625 [==============================] - 16s 26ms/step - loss: 1.2601 - accuracy: 0.5517 - val_loss: 1.4777 - val_accuracy: 0.4931
Epoch 8/100
313/313 [==============================] - 3s 9ms/step - loss: 1.5632 - accuracy: 0.4535
Current Acc:  0.45350000262260437
Best Acc:  0.5781000256538391
0.001 256 512 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 23s 29ms/step - loss: 2.2051 - accuracy: 0.1700 - val_loss: 1.9062 - val_accuracy: 0.2626
Epoch 2/100
625/625 [==============================] - 17s 27ms/step - loss: 1.7619 - accuracy: 0.3272 - val_loss: 1.7549 - val_accuracy: 0.3491
Epoch 3/100
625/625 [==============================] - 17s 27ms/step - loss: 1.5833 - accuracy: 0.4062 - val_loss: 1.6117 - val_accuracy: 0.4185
Epoch 4/100
625/625 [==============================] - 17s 27ms/step - loss: 1.4674 - accuracy: 0.4604 - val_loss: 1.7016 - val_accuracy: 0.4078
Epoch 5/100
625/625 [================

625/625 [==============================] - 19s 30ms/step - loss: 1.5175 - accuracy: 0.4374 - val_loss: 1.7740 - val_accuracy: 0.3677
Epoch 4/100
625/625 [==============================] - 19s 30ms/step - loss: 1.3839 - accuracy: 0.4920 - val_loss: 1.5371 - val_accuracy: 0.4517
Epoch 5/100
625/625 [==============================] - 19s 31ms/step - loss: 1.2889 - accuracy: 0.5375 - val_loss: 1.5398 - val_accuracy: 0.4586
Epoch 6/100
625/625 [==============================] - 19s 31ms/step - loss: 1.2031 - accuracy: 0.5744 - val_loss: 1.4963 - val_accuracy: 0.4948
Epoch 7/100
625/625 [==============================] - 19s 30ms/step - loss: 1.1219 - accuracy: 0.6041 - val_loss: 1.5004 - val_accuracy: 0.4939
Epoch 8/100
625/625 [==============================] - 20s 31ms/step - loss: 1.0569 - accuracy: 0.6253 - val_loss: 1.4564 - val_accuracy: 0.5157
Epoch 9/100
625/625 [==============================] - 19s 30ms/step - loss: 0.9912 - accuracy: 0.6500 - val_loss: 1.1867 - val_accuracy: 0.57

Epoch 7/100
625/625 [==============================] - 19s 30ms/step - loss: 1.1103 - accuracy: 0.6035 - val_loss: 1.5438 - val_accuracy: 0.4866
Epoch 8/100
625/625 [==============================] - 19s 30ms/step - loss: 1.0529 - accuracy: 0.6245 - val_loss: 1.5893 - val_accuracy: 0.4915
Epoch 9/100
625/625 [==============================] - 19s 30ms/step - loss: 0.9933 - accuracy: 0.6473 - val_loss: 1.2726 - val_accuracy: 0.5469
Epoch 10/100
625/625 [==============================] - 19s 30ms/step - loss: 0.9356 - accuracy: 0.6716 - val_loss: 1.3116 - val_accuracy: 0.5736
Epoch 11/100
625/625 [==============================] - 18s 29ms/step - loss: 0.8805 - accuracy: 0.6886 - val_loss: 1.1774 - val_accuracy: 0.6084
Epoch 12/100
625/625 [==============================] - 18s 28ms/step - loss: 0.8345 - accuracy: 0.7100 - val_loss: 1.1504 - val_accuracy: 0.6141
Epoch 13/100
625/625 [==============================] - 18s 29ms/step - loss: 0.7920 - accuracy: 0.7225 - val_loss: 1.1311 - va